 # Project Overview

The domain that I will be working on for this capstone project is image captioning, which is a multi modal task involving both image encoding and language generation. Traditionally CNN-RNN combination has been used for similar tasks. Using this project I want to
implement a project based on the latest research which involves using transformers and has produced state of the art results.
I chose this particular project because it will give me exposure to working on transformers,NLP and Image recognition all together.

## Datasets and Input Data

The dataset that I finally used for this project is Flick 8k because of logistical issues I wasnt able to use Flickr30k as that resulted in my AWS limit getting exceeded.

Flickr8k dataset is a standard benchmark database for image description based tasks was used. The dataset can be found from kaggle. The dataset is divided into 2 parts, first is the 8k image set and second is the csv file containing 3 columns,image_name which is the name of the image file, the comment id and the comment.


# Problem Statement

Generating sentence based image descriptions or Captions for images as input.

## Proposed Solution

We apply a deep learning approach to solving this image captioning problem. We will try to use attention based mechanism to encode the image and generate captions using the decoder.

# Downloading the Data

In [3]:
!pip install kaggle
!pip install kaggle-cli

In [7]:
!chmod 600 /home/ec2-user/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d adityajn105/flickr8k

 99%|█████████████████████████████████████▋| 1.03G/1.04G [00:14<00:00, 73.7MB/s]
100%|██████████████████████████████████████| 1.04G/1.04G [00:14<00:00, 74.3MB/s]


# Analysis